In [ ]:
#using Pkg;
#Pkg.add(url="https://github.com/FerreolS/OptimPackNextGen.jl",rev="verbose")

In [ ]:
include("../src/SphereIFSCalib.jl")

using TwoDimensional, Zygote, StatsBase, Plots, OptimPackNextGen, FITSIO, DelimitedFiles, Tables, DataFrames
using HypothesisTests
plotly()

### Constantes

In [ ]:
# wavelengths
const λ1 = 987.72e-9# laser 1 
const λ2 = 1123.71e-9# laser 2 
const λ3 = 1309.37e-9# laser 3
#const λ4 = 1545.10e-9  # laser 4  
λlaser = [λ1,λ2,λ3]
λ0 = mean(λlaser);# reference
wavelengthrange = LinRange(850e-9,1600e-9,50); # coarse wavelength range of the instrument

## Ensemble des lenslets:

In [ ]:
#Bbox of the lenslet: box autour d'un set de 3 gaussiennes
bbox = BoundingBox(xmin=1012, ymin=1048, xmax=1018, ymax=1092);

a0 = [300.0 , 400.0, 500.0];
fwhm0 = [2.0, 2.0 , 2.0];

#il manque les coeff C0 pour pouvoir generer le modele

### 1 lecture des fichiers txt

On veut recuperer en sortie de fonction la premiere colonne (c0, soit le centre de la lenslet). Plus tard, ce sera l'ensemble du fichier pour traiter le cas broadlamp.

In [ ]:
coeffx = readdlm("/home/user/stage/HR_4796-HD_95086/Calibration_wave_spec/coef_pol_x.txt", header = false)
cx0 = coeffx[:,1];
#cx1 = f[:,2];
#cx2 = f[:,3];

coeffy = readdlm("/home/user/stage/HR_4796-HD_95086/Calibration_wave_spec/coef_pol_y.txt", header = false)
cy0 = coeffy[:,1];
#cy1 = cy[:,2];
#cy2 = cy[:,3];

#round.(Int, cx0); #conversion de float en int
#round.(Int, cy0);

### 2 ouvrir et trouver positions dans l'image brute

In [ ]:
#1/ IFS_sim_wave.fits: on ouvre le modele, qui a peu de BP, donc les fit des lenslets seront bons. f5 = FITS("/home/user/Pictures/IFS_sim_wave.fits");
#2/ IFS_calib_wave_corrected: "vrai" fichier de depart
f5 = FITS("/home/user/Pictures/IFS_calib_wave_corrected.fits", "r");

In [ ]:
#Read the data or a subset thereof from disk, and save it in a pre-allocated output array A.
A = Array{Float64}(undef,2048,2048); #pre-allocate matrix A
read!(f5[1], A);

In [ ]:
close(f5);

In [ ]:
#On parcours les lenslets
laser =  LaserModel(λlaser,a0,fwhm0);
lmod = LensletModel(λ0,laser.nλ-1,bbox);

nb_fit = 5 #test sur quelques lenslets
for i=1:nb_fit 
    C0 = zeros(2,laser.nλ)
    C0[1:2,1:2] = [ [cy0[i,1] 45]; [cx0[i,1] 22] ]
    UpdateDispModel(lmod.dmodel,C0) #Update the coefficients  of the DispModel.
    
    laser =  LaserModel(λlaser,a0,fwhm0)
    lenslet = LensletModel(λ0,laser.nλ-1,round(bbox))

    wxyinit = lmod.dmodel.(wavelengthrange)
    (wxinit,wyinit) = collect.( zip(wxyinit...))
end


In [ ]:
#loi de dispersion
wxy0 = lmod.dmodel.(wavelengthrange);
(wx0,wy0) = collect.( zip(wxy0...));

In [ ]:
lmod.dmodel.(λlaser) 
ldata = view(A, bbox);

### 3 affichage

In [ ]:
#heatmap(ldata, title = "Fit laser")
heatmap(clamp.(A,0,1500), title = "Fit laser")
plot!(wy0,wx0; label="dispersion law")

In [ ]:
#Vecteur: optimisation plus rapide car les trois coefficients a traiter sont donnes en meme temps
function  likelihood(x::Vector{Float64}) 
    (a,fwhm,c) = (x[1:(laser.nλ)],x[(laser.nλ+1):(2*laser.nλ)],reshape(x[(2*laser.nλ+1):(4*laser.nλ)],2,:));
    UpdateDispModel(lmod.dmodel, c);
    UpdateLaserModel(laser,a,fwhm);
    return sum((ldata .- LensletLaserImage(lmod,laser)).^2) # multiplier par carte des BP pour ne pas les prendre en compte dans le fit
end

In [ ]:
x0 = vcat([a0[:],fwhm0[:],C0[:] ]...) #concatenation de a0, fwhm0 et C0

@show cost = likelihood(x0)
∇cost = gradient(likelihood,x0);

### Optimisation

In [ ]:
#Parametres d'initialisation
ainit = [300.0 , 400.0, 500.0];#a0  .+ (rand(Float64,laser.nλ) .- 0.5);
fwhminit = [2.0, 2.0 , 2.0]; #fwhm0  .+ (rand(Float64,laser.nλ) .- 2);
cinit = zeros(2,laser.nλ)
cinit[1:2,1:3] = [[ 4.0 0 0]; [21 -80 0]] 

#UpdateDispModel(lmod.dmodel, cinit);
#UpdateLaserModel(laser,ainit,fwhminit);

laser =  LaserModel(λlaser,a0,fwhm0)
lenslet = LensletModel(λ0,laser.nλ-1,round(bbox))

wxyinit = lmod.dmodel.(wavelengthrange)
(wxinit,wyinit) = collect.( zip(wxyinit...))

xinit = vcat([ainit[:],fwhminit[:],cinit[:]]...)

∇cost = gradient(likelihood,xinit )

lkl = LikelihoodIFS(lenslet,laser,view(data,lenslet.bbox))
xopt = vmlmb(lkl, xinit; verb=50);

In [ ]:
#Parametres d'optimisation
(aopt,fwhmopt,copt) = (xopt[1:(laser.nλ)],xopt[(laser.nλ+1):(2*laser.nλ)],reshape(xopt[(2*laser.nλ+1):(4*laser.nλ)],2,:));
UpdateDispModel(lmod.dmodel, copt);
UpdateLaserModel(laser,aopt,fwhmopt);

In [ ]:
@show(ainit,aopt);

@show(fwhminit,fwhmopt);

@show(cinit,copt);

In [ ]:
#Point de depart de l'optimiseur
UpdateDispModel(lmod.dmodel, cinit);
UpdateLaserModel(laser,ainit,fwhminit);
limageinit = LensletLaserImage(lmod,laser)

#heatmap(limageinit, title = "Initialisation")

In [ ]:
#Point d'arrivee de l'optimiseur
UpdateDispModel(lmod.dmodel, copt);
UpdateLaserModel(laser,aopt,fwhmopt);
limageopt = LensletLaserImage(lmod,laser)

#heatmap(limageopt, title = "Point d'arrivee de l'optimiseur")


In [ ]:
#Residus
#heatmap( ldata .- limageopt, title = "Residus")

### 4 sauvegarder output (position)

ce qu'on veut garder: lmod sous forme de vecteur, 1 modele de lenslet par lenslet

In [ ]:
(aopt,fwhmopt,copt) = (xopt[1:(laser.nλ)],xopt[(laser.nλ+1):(2*laser.nλ)],reshape(xopt[(2*laser.nλ+1):(4*laser.nλ)],2,:));

Ajouter un flag de qualité (bon/mauvais ajustements)
Histogramme des valeurs de χ2, pour voir si on peut enlever des outliers
Histogramme des parametres du fits (a, fwhm, c)

In [ ]:
#Si loi trop eloignee, envoyer un warning

In [ ]:
#hypothèse nulle H0:

ChisqTest(lmod)

### Critère de réjection des mauvais ajustement

In [ ]:
# Boucle sur vecteur lmod: for i=1:eof
# si pas de gaussiennes dans residus ( = si point arrivee opt a 3 spots fittes), alors fit bon
# sinon, fit mauvais
